# Práctica 4 Reconocimiento de Formas: Clasificador Bayesiano Paramétrico

* **Alumno 1**: Javier Barragán Haro
* **Alumno 2**: Victor Nieves Sanchez


In [23]:

import numpy as np
from sklearn.covariance import ShrunkCovariance
from sklearn import preprocessing
from abc import abstractmethod
import pandas as pd
import numpy as np


class Classifier:

    @abstractmethod
    def fit(self,X,y):
        pass

    @abstractmethod
    def predict(self,X):
        pass

class ClassifBayesianoParametrico(Classifier):
    def __init__(self, share_covs=False, shrinkage=0.0):
        """Constructor de la clase
        share_covs: Indica si la matriz de covarianzas va a ser compartida entre las distintas clases.
        shrinkage: Parámetro que determina la diagonalidad de la matriz de covarianzas. Ver sklearn.covariance.ShrunkCovariance
        """
        assert 0 <= shrinkage <= 1
        self.labels = None
        self.ln_apriories = None
        self.means = None
        self.ln_determinants = None
        self.inv_covs = None
        self.share_covs = share_covs
        self.shrinkage = shrinkage
        self.scaler = preprocessing.StandardScaler()

    def fit(self, X, y):
        """Entrena el clasificador
        X: matriz numpy cada fila es un dato, cada columna una medida
        y: vector de etiquetas, tantos elementos como filas en X
        retorna objeto clasificador"""
        assert X.ndim == 2 and X.shape[0] == len(y)
        # Aseguramos que las etiquetas son numeros tal que: [0, 1, ..., N]
        y = pd.factorize(y)[0]
        self.labels = y
        
        # Preprocesamos los datos de entrada
        X = self.scaler.fit_transform(X)    
        
        # Contar cuantos ejemplos hay de cada etiqueta
        unique, counts = np.unique(y, return_counts=True)
        
        self.ln_apriories = np.array([np.log((counts[i]/np.sum(counts))) for i in unique])
        
        # Calcular para los ejemplos de cada clase, la media de cada una de sus características (centroide)
        self.means = np.array([np.mean(X[y==i], axis=0) for i in unique])
        
        if self.share_covs:
            # Restamos al dato de cada clase su centroide
            X_mean0 = np.array([X[y==i] - self.means[i] for i in unique])
            # Calcula la matriz de covarianzas empleando la clase ShrunkCovariance de sklearn
            cov = np.array([ShrunkCovariance(shrinkage=self.shrinkage).fit(X_mean0[i]).covariance_ for i in unique ])
            # La reproducimos tantas veces como número de clases
        else:
            # Calcula la matriz de covarianzas empleando la clase ShrunkCovariance de sklearn
             cov = np.array([ShrunkCovariance(X,shrinkage=self.shrinkage).fit(X_mean0[i]).covariance_ for i in unique]) 
        # Para cada una de las clases, calcular el logaritmo neperiano de su matriz de covarianzas (puedes emplear compresión de listas o la función map)
        self.ln_determinants = np.log(list(map(np.linalg.det, cov)))
        # Para cada una de las clases, calcular la inversa de su matriz de covarianzas (puedes emplear compresión de listas o la función map)
        self.inv_covs = np.array(list(map(np.linalg.pinv,cov)))
        
        return self

    def predict(self, X):
        """Estima el grado de pertenencia de cada dato a todas las clases
        X: matriz numpy cada fila es un dato, cada columna una medida del vector de caracteristicas.
        Retorna una matriz, con tantas filas como datos y tantas columnas como clases tenga
        el problema, cada fila almacena los valores pertenencia de un dato a cada clase"""
        assert self.means is not None, "Error: The classifier needs to be fitted. Please call fit(X, y) method."
        assert X.ndim == 2 and X.shape[1] == self.means.shape[1]

        # Preprocesamos nuestros datos
        X = self.scaler.fit_transform(X)        

        # Resta la media de cada clase a cada ejemplo en X
        X_mean0 = self.means[:,np.newaxis,:] - X 
        
        # Calcula el logaritmo de la función de decisión gausiana
        # Transparencias de Métodos paramétricos de clasificación: página 14:
        # -(1/2)ln|Sigma_i| - (1/2)*(x- mu_i)^T Sigma_i^-1 (x- mu_i) + lnP(alpha_i)
        return -0.5 * self.ln_determinants[:, np.newaxis] \
            - 0.5 * np.array([np.sum((X_mean0[i] @ self.inv_covs[i]) 
                                     * X_mean0[i], axis=1) for i in np.unique(y)]) \
            + self.ln_apriories[:, np.newaxis]
    def pred_label(self, X):
        """Estima la etiqueta de cada dato. La etiqueta puede ser un entero o bien un string.
        X: matriz numpy cada fila es un dato, cada columna una medida
        retorna un vector con las etiquetas de cada dato"""
        return np.argmax(X, axis = 0)
    
    def num_aciertos(self, X): 
        """Cuenta el numero de aciertos del clasificador para un conjunto de datos X.
        X: matriz de datos a clasificar"""
        same_values = []
        [same_values.append(X[i] == self.labels[i]) for i in range(0, len(self.labels))]
        number = same_values.count(True)

        return number, (number / len(X)) * 100  

## Iris Dataset
Carga, entrenamiento, predicción y evaluación en la base de datos de Iris:

In [14]:
# 1. Cargar los datos de la base de datos de entrenamiento
from sklearn.datasets import load_iris
from sklearn.covariance import ShrunkCovariance
dataset = load_iris()
X = dataset.data
# print("X: \n" + str(X))
y = dataset.target
# print("y: \n" + str(y))
# # 2. Entrenar el clasificador
ClassifBayesianoParametrico = ClassifBayesianoParametrico(True,0.01)

ClassifBayesianoParametrico.fit(X, y)

predict_matrix = ClassifBayesianoParametrico.predict(X)

labels_matrix = ClassifBayesianoParametrico.pred_label(predict_matrix)

correct = ClassifBayesianoParametrico.num_aciertos(labels_matrix)
# # 3. Predecir empleando la base de datos de entrenamiento (X)
print(correct)
# # 4. Evaluar el clasificador calculando el porcentaje de datos correctamente clasificados


(147, 98.0)


In [15]:
# 1. Cargar los datos de la base de datos de entrenamiento
from sklearn.datasets import load_iris
dataset = load_iris()
X = dataset.data
# print("X: \n" + str(X))
y = dataset.target




## Wine dataset
Carga, entrenamiento, predicción y evaluación en la base de datos de Wine:

In [18]:
# 1. Cargar los datos de la base de datos de entrenamiento
from sklearn.datasets import load_wine
dataset = load_wine()
X = dataset.data
# print("X: \n" + str(X))
y = dataset.target
# print("y: \n" + str(y))
ClassifBayesianoParametrico = ClassifBayesianoParametrico(True,0.01)

ClassifBayesianoParametrico.fit(X, y)

predict_matrix = ClassifBayesianoParametrico.predict(X)

labels_matrix = ClassifBayesianoParametrico.pred_label(predict_matrix)

correct = ClassifBayesianoParametrico.num_aciertos(labels_matrix)
# 2. Entrenar el clasificador
print(correct)
# 3. Predecir empleando la base de datos de entrenamiento (X)

# 4. Evaluar el clasificador calculando el porcentaje de datos correctamente clasificados



(177, 99.43820224719101)


## Breast cancer dataset
Carga, entrenamiento, predicción y evaluación en la base de datos de Breast cancer:

In [20]:
# 1. Cargar los datos de la base de datos de entrenamiento
from sklearn.datasets import load_breast_cancer
dataset = load_breast_cancer()
X = dataset.data
# print("X: \n" + str(X))
y = dataset.target
# print("y: \n" + str(y))
ClassifBayesianoParametrico = ClassifBayesianoParametrico(True,0.01)

ClassifBayesianoParametrico.fit(X, y)

predict_matrix = ClassifBayesianoParametrico.predict(X)

labels_matrix = ClassifBayesianoParametrico.pred_label(predict_matrix)

correct = ClassifBayesianoParametrico.num_aciertos(labels_matrix)
print(correct)
# 2. Entrenar el clasificador

# 3. Predecir empleando la base de datos de entrenamiento (X)

# 4. Evaluar el clasificador calculando el porcentaje de datos correctamente clasificados


(551, 96.8365553602812)


## MNIST Database (Modified National Institute of Standards and Technology database)
MNIST es una base de datos de texto manuscrito, que se usa de forma clásica para entrenar sistemas de procesado de imágenes.

Carga, entrenamiento, predicción y evaluación en la base de datos de MNIST:

In [26]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml

# Cargamos MNIST desde internet ( https://www.openml.org/d/554 )
all_X, all_y = fetch_openml('mnist_784', version=1, return_X_y=True, cache=True)
X = all_X[:60000]
y = all_y[:60000]
y = pd.factorize(y)[0]
print(y)

# print("X: \n" + str(X))
# print("y: \n" + str(y))

[0 1 2 ... 0 7 9]


In [27]:
# 2. Entrenar el clasificador
pClassifBayesianoParametrico = ClassifBayesianoParametrico(True,0.32)

ClassifBayesianoParametrico.fit(X, y)

predict_matrix = ClassifBayesianoParametrico.predict(X)

labels_matrix = ClassifBayesianoParametrico.pred_label(predict_matrix)

correct = ClassifBayesianoParametrico.num_aciertos(labels_matrix)
print(correct)
# 3. Predecir empleando la base de datos de entrenamiento (X)

# 4. Evaluar el clasificador calculando el porcentaje de datos correctamente clasificados


TypeError: fit() missing 1 required positional argument: 'y'

## Isolet Dataset (Isolated Letter Speech Recognition)
Carga, entrenamiento, predicción y evaluación en la base de datos de Isolet

In [28]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml

# 1. Cargar los datos de la base de datos de entrenamiento
X, y = fetch_openml('isolet', version=1, return_X_y=True, cache=True)
y = pd.factorize(y)[0]
# print("X: \n" + str(X))
# print("y: \n" + str(y))
ClassifBayesianoParametrico = ClassifBayesianoParametrico(True,0.01)

ClassifBayesianoParametrico.fit(X, y)

predict_matrix = ClassifBayesianoParametrico.predict(X)

labels_matrix = ClassifBayesianoParametrico.pred_label(predict_matrix)

correct = ClassifBayesianoParametrico.num_aciertos(labels_matrix)
print(correct)

C:\Users\javic\Anaconda3\lib\site-packages\ipykernel_launcher.py:66: RuntimeWarning: divide by zero encountered in log


(300, 3.8476337052712584)


In [8]:
# 2. Entrenar el clasificador

# 3. Predecir empleando la base de datos de entrenamiento (X)

# 4. Evaluar el clasificador calculando el porcentaje de datos correctamente clasificados


Resultados de los tres experimentos:

| Base de datos | Número de aciertos | Porcentaje de aciertos |
| --- | --- | --- |
| Iris   |  |  |
| Wine   |  |  |
| Cancer |  |  |
| MNIST  |  |  |
| Isolet |  |   |